Prepare the dataset and train a model on the data.

In [1]:
import os
import sys
project_root = os.path.abspath(os.path.join(os.getcwd(), "../../.."))
sys.path.append(project_root)
from examples.mia.text_mia.utils.tabds_data_preparation import *
from examples.mia.text_mia.utils.tabds_model_preparation import *

/home/amandus/miniconda3/envs/leakpro/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print(os.path.join(os.getcwd(), "tab_data/"))

/home/amandus/msc-code/LeakPro/examples/mia/text_mia/tab_data/


In [3]:
path = os.path.join(os.getcwd(), "tab_data/")
dataset = preprocess_tab_dataset(path)

Loading local dataset.
Local dataset succsesfully loaded.


In [3]:
n_classes = 3 # check a nice way to set this. Maybe it's better to do with yaml file
# Maybe not needed if loading models tho, if this is parameters used to construct model. 
train_loader, test_loader = get_tab_dataloaders(dataset, train_fraction=0.1, test_fraction=0.1)


In [4]:
n_classes = 3 # check a nice way to set this. Maybe it's better to do with yaml file
# Maybe not needed if loading models tho, if this is parameters used to construct model. 
train_loader, test_loader = get_tab_dataloaders(dataset, train_fraction=0.1, test_fraction=0.1)

# Train the model
if not os.path.exists("target"):
    os.makedirs("target")
model = TABBERT(pt_model= "allenai/longformer-base-4096", num_classes=n_classes)
train_acc, train_loss, test_acc, test_loss = create_trained_model_and_metadata(model, 
                                                                               train_loader, 
                                                                               test_loader, 
                                                                               epochs=1)

Training
Epoch 1 started


100%|██████████| 20/20 [00:13<00:00,  1.51it/s]


Evaluating


100%|██████████| 20/20 [00:03<00:00,  6.59it/s]


Create LeakPro objects and run attack

In [6]:
from tabds_handler import TABInputHandler

from leakpro import LeakPro

# Read the config file
config_path = "audit.yaml"

# Prepare leakpro object
leakpro = LeakPro(TABInputHandler, config_path)

# Run the audit 
leakpro.run_audit()

2024-11-07 12:14:23,186 INFO     Target model blueprint created from TABBERT in utils/tabds_model_preparation.py.
2024-11-07 12:14:23,189 INFO     Loaded target model metadata from ./target/model_metadata.pkl


2024-11-07 12:14:25,475 INFO     Loaded target model from ./target
2024-11-07 12:14:46,123 INFO     Loaded population dataset from tab_data/tab_train_200_dataset.pkl
2024-11-07 12:14:46,125 INFO     Loaded population dataset from tab_data/tab_train_200_dataset.pkl
2024-11-07 12:14:46,127 INFO     Configuring RMIA attack
2024-11-07 12:14:46,128 INFO     Added attack: rmia
2024-11-07 12:14:46,129 INFO     Preparing attack: rmia
2024-11-07 12:14:46,129 INFO     Preparing shadow models for RMIA attack
2024-11-07 12:14:46,130 INFO     Preparing attack data for training the RMIA attack
2024-11-07 12:14:46,132 INFO     Check for 3 shadow models (dataset: 203 points)
2024-11-07 12:14:46,135 INFO     Number of existing models exceeds or equals the number of models to create
2024-11-07 12:14:46,136 INFO     Loading shadow model 0


[ 32 131  33  23 196  53 164 144 165 139 124 115 101  83 167 175 192 152
 148 190  88  44  49 187 151 191  48  30 137 188  54 132 122 104  65  61
  43 182 113 123 145   9  75  82  86 147 107 157 114  69  96 120 171  66
  52 134  31 143 111 184   6 170  68 138  13 168  51   8  89 146  84 110
 155 100  10 180  90 130  79  94  78  40  59 163 140   4  12  81  47  18
  50 109  19 194  35 153 186  14  57  20  26 174  76 102 202  99   3  71
 162  73 199  21 133  46   7 156 185  62 177 198 197  98 178 106 183  39
 149  56  15 126 195  97  63 117  36 150 125 201  34  92 135  42  38  91
  45   1  16 181 169  70 141 176  74 159 112  77 200 154  60 142 121  64
  93 193 105  41 160  55  25  22 108 119   0  17 173  85 158  58   5 129
  80  24  95  67 136   2  87  29 116  27 172 118  11 166  28 103 128 161
  72 127 179 189  37]


2024-11-07 12:14:48,192 INFO     Loaded model from ./leakpro_output/attack_objects/shadow_model/shadow_model_0.pkl
2024-11-07 12:14:48,195 INFO     Loading shadow model 2
2024-11-07 12:14:50,183 INFO     Loaded model from ./leakpro_output/attack_objects/shadow_model/shadow_model_2.pkl
2024-11-07 12:14:50,185 INFO     Loading shadow model 1
2024-11-07 12:14:52,383 INFO     Loaded model from ./leakpro_output/attack_objects/shadow_model/shadow_model_1.pkl
2024-11-07 12:14:52,385 INFO     Running attack: rmia
2024-11-07 12:14:59,707 INFO     Running RMIA online attack
2024-11-07 12:14:59,709 INFO     Loading metadata 0
2024-11-07 12:14:59,711 INFO     Loading metadata 2
2024-11-07 12:14:59,714 INFO     Loading metadata 1
2024-11-07 12:15:35,269 INFO     Number of points in the audit dataset that are used for online attack: 18


IndexError: shape mismatch: indexing arrays could not be broadcast together with shapes (18,) (18,4096) 